In [1]:
import pandas as pd
import glob 
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
from datetime import datetime
import pickle
from dateutil import tz

def find_classification(max_wspd, shear):
    '''
    Find the LLJ classification based on these values.
    
    From Vanderwende: 
            Speed    Shear
    LLJ-0:   10        5
    LLJ-1:   12        6
    LLJ-2:   16        8
    LLJ-3:   20       10
    '''
    
    if ((max_wspd>=20) and (shear>=10)):
        return 3
    elif ((max_wspd>=16) and (shear>=8)):
        return 2
    elif ((max_wspd>=12) and (shear>=6)):
        return 1
    else:
        return 0
    
def datestr(num,format="%Y-%m-%d %H:%M:%S.%f"):
    from datetime import datetime
    string=datetime.utcfromtimestamp(num).strftime(format)
    return string

    
height_thresh = 700 # threshold for highest nose height to accept
ws_thresh = 10
shear_thresh = 5

In [2]:
data_save_path = "./data_siteH_VAD/"
fig_save_path = "./plots/LLJ_plots/"
filenames = glob.glob(data_save_path +'*')

# set up output columns and dataframe
Time = []
time_local = []
LLJ_class = []
nose_heights = []
sfc_nose_shr = []
above_nose_shr = []
nose_wd = []
nose_ws = []


for i, filename in enumerate(filenames[:]):

    print("Processing file: {}/{}".format(i,len(filenames)))

    file = open(filename,'rb')
    df = pickle.load(file)
    file.close()

    timestamp = df.time.values[0]
    Time.append(timestamp)

    to_zone = tz.gettz('America/Chicago')
    local_time = pd.Timestamp(timestamp).tz_localize("UTC").tz_convert(to_zone)
    print(local_time, local_time.to_datetime64(),timestamp) 
    time_local.append(local_time.to_datetime64())

    summary = pd.DataFrame()

    if len(df.windspeed.dropna()) < 35:
                LLJ_class.append(np.nan)
                nose_heights.append(np.nan)
                sfc_nose_shr.append(np.nan)
                above_nose_shr.append(np.nan)
                nose_wd.append(np.nan)
                nose_ws.append(np.nan)
                continue
    
    # look for isolated values with NaNs
    # nan_idxs = np.where(df['windspeed'].isnull())[0]

    # find max windspeed and shear above nose
    ws_max = df.windspeed.max()
    ws_max_idx = df.windspeed.argmax()
    shear_above = np.ptp(df.windspeed[ws_max_idx:].dropna().values)
    height = df.altitude[ws_max_idx]
    # Check LLJ conditions
    if (ws_max > ws_thresh) and (shear_above > shear_thresh) and (height < height_thresh):
        shear_below = np.ptp(df.windspeed[:ws_max_idx].dropna().values)
        class_ = find_classification(ws_max, shear_above)
        # add values to columns
        LLJ_class.append(class_)
        nose_heights.append(height)
        sfc_nose_shr.append(shear_below)
        above_nose_shr.append(shear_above)
        nose_wd.append(df.wind_direction[ws_max_idx])
        nose_ws.append(ws_max)    
    else:
        class_ = np.nan
        LLJ_class.append(np.nan)
        nose_heights.append(np.nan)
        sfc_nose_shr.append(np.nan)
        above_nose_shr.append(shear_above)
        nose_wd.append(np.nan)
        nose_ws.append(ws_max)

    # Add columns to the dataframe
    # times = [datestr(tt) for tt in df.time.values]
    # summary['UTC Time (unix)'] = df.time.values
    # summary['UTC Time'] = times
    # summary['LLJ class'] = LLJ_class
    # summary['Nose Windspeed [m/s]'] = nose_ws
    # summary['Nose Height [m]'] = nose_heights
    # summary['Surface to nose shear [m/s]'] = sfc_nose_shr
    # summary['Shear above nose [m/s]'] = above_nose_shr
    # summary['Nose Wind Direction [degrees]'] = nose_wd
    # # concatenate with the rest of the data
    # summary_full = pd.concat([summary_full, summary])
    # print(file, 'processed')

    # print(shear_above, class_)
    
    plt.figure()
    plt.plot(df["windspeed"], df["altitude"], marker="+")
    plt.ylabel("AGL Altitude [m]")
    plt.xlabel("Windspeed [m/s]")
    plt.scatter(ws_max,df.altitude[ws_max_idx], alpha = 0.5)
    plt.title("LLJ class: {}".format(class_))
    
    plt.savefig(fig_save_path + filename.split("/")[-1].split(".")[0] + ".png",
                format = "png",
                dpi=500)
    plt.close()



summary_df = pd.DataFrame(
    list(
    zip(Time,
        LLJ_class, 
        nose_heights, 
        sfc_nose_shr,
        above_nose_shr,
        nose_ws,
        nose_wd)),
    columns = [
            "Time (UTC)",
            "LLJ class", 
            "Nose Height [m]",
            "Surface to nose shear [m/s]", 
            "Shear above nose [m/s]",
            "Nose Windspeed [m/s]",
            "Nose Wind Direction [rad]"
            ],      
)


summary_df.to_pickle("./data_siteH_LLJs/LLJ_summary.pkl")


summary_full = summary_df.set_index(summary_df['Time (UTC)']).sort_index(ascending=True).drop(columns=['Time (UTC)'])

summary_full.to_excel("./data_siteH_LLJs/LLJ_summary.xlsx")

Processing file: 0/354
2022-11-12 13:00:20.379638-06:00 2022-11-12T19:00:20.379638000 2022-11-12T19:00:20.379638000
Processing file: 1/354
2022-11-12 03:00:20.980453-06:00 2022-11-12T09:00:20.980453000 2022-11-12T09:00:20.980453000
Processing file: 2/354
2022-11-13 14:00:20.736694-06:00 2022-11-13T20:00:20.736694000 2022-11-13T20:00:20.736694000
Processing file: 3/354
2022-11-11 09:00:20.908355-06:00 2022-11-11T15:00:20.908355000 2022-11-11T15:00:20.908355000
Processing file: 4/354
2022-11-10 23:00:20.549583-06:00 2022-11-11T05:00:20.549583000 2022-11-11T05:00:20.549583000
Processing file: 5/354
2022-11-11 20:30:03.270149-06:00 2022-11-12T02:30:03.270149000 2022-11-12T02:30:03.270149000
Processing file: 6/354
2022-11-17 07:30:03.141403-06:00 2022-11-17T13:30:03.141403000 2022-11-17T13:30:03.141403000
Processing file: 7/354
2022-11-16 15:00:20.626831-06:00 2022-11-16T21:00:20.626831000 2022-11-16T21:00:20.626831000
Processing file: 8/354
2022-11-13 22:00:20.729827-06:00 2022-11-14T04:00